In [1]:
import os

In [2]:
%pwd

'd:\\UTD\\Academics\\Kidney-Disease-Classification-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\UTD\\Academics\\Kidney-Disease-Classification-Project'

In [6]:
%pip install dagshub mlflow

import dagshub
dagshub.init(repo_owner='samad-mehndi', repo_name='Kidney-Disease-Classification-Project', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Initialized MLflow to track repo "samad-mehndi/Kidney-Disease-Classification-Project"

Repository samad-mehndi/Kidney-Disease-Classification-Project initialized!

🏃 View run tasteful-trout-398 at: https://dagshub.com/samad-mehndi/Kidney-Disease-Classification-Project.mlflow/#/experiments/0/runs/e10a096a262c407c86da058cbfb61f4f
🧪 View experiment at: https://dagshub.com/samad-mehndi/Kidney-Disease-Classification-Project.mlflow/#/experiments/0


In [7]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/samad-mehndi/Kidney-Disease-Classification-Project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="samad-mehndi"

In [8]:

import tensorflow as tf

In [9]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [11]:

import sys
import os

# Add the 'src' directory to the path
sys.path.append(os.path.join(os.getcwd(), 'src'))

from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [ ]:
class configurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/dataset_ct_scan",
            mlflow_uri="https://dagshub.com/samad-mehndi/Kidney-Disease-Classification-Project.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [13]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [14]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [15]:

try:
    config = configurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

Found 2207 images belonging to 2 classes.


C:\Users\SAMAD MEHNDI\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


138/138 ━━━━━━━━━━━━━━━━━━━━ 157s 1s/step - accuracy: 0.9174 - loss: 0.1921


2025/05/29 16:10:45 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/05/29 16:10:56 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2025/05/29 16:11:17 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 2
Created version '2' of model 'VGG16Model'.


🏃 View run honorable-fawn-649 at: https://dagshub.com/samad-mehndi/Kidney-Disease-Classification-Project.mlflow/#/experiments/0/runs/46f2510d2b6e46209d29ea5d2d91dc06
🧪 View experiment at: https://dagshub.com/samad-mehndi/Kidney-Disease-Classification-Project.mlflow/#/experiments/0
